In [4]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-220264
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-220264


In [19]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "hyper-drive-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Following based on Lesson 'The Azure ML SDK' Exercise 8
# Verify that the cluster does not exist already
try:
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_D2_V2', 
        max_nodes=4
    )

    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
# Arguments specified in train.py
# C (float): Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
# max_iter (int): Maximum number of iterations taken for the solvers to converge.
ps = RandomParameterSampling( {
    "--C" : uniform(0.1,1),
    "--max_iter" : choice(50, 100, 150, 200)
    }
)

# Specify a Policy
policy = BanditPolicy(
    evaluation_interval=1, 
    slack_factor=0.1,
    delay_evaluation=4
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
# This is the estimator for the train script
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=compute_cluster,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4
)

In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(
    config=hyperdrive_config,
    show_output=True
)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_138d5ce1-3ca7-4c2b-91c3-234b1079c5dd
Web View: https://ml.azure.com/runs/HD_138d5ce1-3ca7-4c2b-91c3-234b1079c5dd?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-220264/workspaces/quick-starts-ws-220264&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-12-26T18:08:07.908224][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-12-26T18:08:10.3937554Z][SCHEDULER][INFO]Scheduling job, id='HD_138d5ce1-3ca7-4c2b-91c3-234b1079c5dd_0' 
[2022-12-26T18:08:10.5485480Z][SCHEDULER][INFO]Scheduling job, id='HD_138d5ce1-3ca7-4c2b-91c3-234b1079c5dd_1' 
[2022-12-26T18:08:10.651908][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2022-12-26T18:08:10.6691383Z][SCHEDULER][INFO]Scheduling job, id='HD_138d5ce1-3ca7-4c2b-91c3-234b1079c5dd_2' 
[2022-12-26T18:08:10.7147454Z][SCHEDULER][INFO]Scheduling job, id='HD_138d5ce1-3ca7-4c2b-91c3-234b1079c5

{'runId': 'HD_138d5ce1-3ca7-4c2b-91c3-234b1079c5dd',
 'target': 'hyper-drive-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-12-26T18:08:07.008787Z',
 'endTimeUtc': '2022-12-26T18:12:13.176342Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '55d33fa5-e054-4f44-ad67-6e363f9a2a7e',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': 'infinite_space_size',
  'score': '0.9091047040971169',
  'best_child_run_id': 'HD_138d5ce1-3ca7-4c2b-91c3-234b1079c5dd_3',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_138d5ce1-3ca7-4c2b-91c3-234b1079c5dd_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telem

In [19]:
import joblib
# Get your best run and save the model from that run.
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()

best_hyperdrive_run_metrics = best_hyperdrive_run.get_metrics()
hyperdrive_parameter_values = best_hyperdrive_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_hyperdrive_run.id)
print('\n Accuracy:', best_hyperdrive_run_metrics['Accuracy'])
print('\n Hyperparameters:', hyperdrive_parameter_values)

# Use joblib for saving the model
print('Best run file names: ', best_hyperdrive_run.get_file_names())
#joblib.dump(best_hyperdrive_run, 'best_hyperdrive_model.joblib')

Best Run Id:  HD_138d5ce1-3ca7-4c2b-91c3-234b1079c5dd_3

 Accuracy: 0.9091047040971169

 Hyperparameters: ['--C', '0.9711268966683734', '--max_iter', '100']
Best run file names:  ['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2022-12-26-18', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [1]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(
        path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv',
)

# Using a TabularDatasetFactory method to split data into training and test
training_data, test_data = ds.random_split(
        percentage=0.8
)

In [2]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(training_data)

train_data_cleaned = x.join(y)

In [3]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=25,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data_cleaned,
    label_column_name='y',
    n_cross_validations=3
)

In [5]:
# Submit your automl run

automl_exp = Experiment(ws, 'automl_experiment')
automl_run = automl_exp.submit(
    config=automl_config,
    show_output=True
)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_2048ad9c-5710-4b0a-8353-cb9c44a1fee3,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.


2022/12/26 19:32:57 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2929                          |1                               |26276                                 |
+------------------------------+--------------------------------+-----

2022-12-26:19:55:28,78 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2022-12-26:19:55:29,525 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2022-12-26:19:55:29,527 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2022-12-26:19:55:31,3 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2022-12-26:19:55:31,4 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may cha

StackEnsemble                                  0:00:41             0.9166    0.9175
Stopping criteria reached at iteration 32. Ending experiment.
********************************************************************************************
Current status: BestRunExplainModel. Best run model explanations started
Current status: ModelExplanationDataSetSetup. Model explanations data setup completed
Current status: PickSurrogateModel. Choosing LightGBM as the surrogate model for explanations
Current status: EngineeredFeatureExplanations. Computation of engineered features started


2022-12-26:19:57:39,530 INFO     [explanation_client.py:334] Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
********************************************************************************************


NameError: name 'RunDetails' is not defined

In [10]:
# Retrieve your best automl model.

import joblib
# Get your best run and save the model from that run.
best_automl_run, fitted_model = automl_run.get_output()

print('Best run: ', best_automl_run)
print('\n Model: ', fitted_model)
print('\n Best run details: ', best_automl_run.get_details())


Best run:  Run(Experiment: automl_experiment,
Id: AutoML_2048ad9c-5710-4b0a-8353-cb9c44a1fee3_30,
Type: None,
Status: Completed)

 Model:  Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('4', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.6933333333333332, learning_rate=0.09473736842105263, max_bin=110, max_depth=8, min_child_weight=6, min_data_in_leaf=0.003457931034482759, min_split_gain=1, n_estimators=25, n_jobs=1, num_leaves=227, problem_info=

In [17]:
# Save your best automl model

joblib.dump(fitted_model, 'fitted_model.joblib')

['fitted_model.joblib']

In [ ]:
# Delete compute instance
compute_cluster.delete()